In [97]:
from sklearn.datasets import load_wine
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import nn
import numpy as np
import pandas as pd

In [2]:
# Activation functions
def sigmoid(x):
    return 1/(1 + np.exp(-x))

# Derivative of tanh from its output
def dsig(y):
    return y*(1 - y)

In [39]:
# The neural network framework
class Layer:
    def __init__(self, num_inputs, num_outputs):
    # Init all weights between [-1 .. 1].
    # Each input is connected to all outputs.
    # One line per input and one column per output.
        self.weights = np.random.uniform(-1, 1, (num_inputs, num_outputs))

    def forward(self, input):
        self.output = sigmoid(input.dot(self.weights))
        return self.output

    def computeGradient(self, error):
        self.delta = error * dsig(self.output)
        # Returns the gradient
        return self.delta.dot(self.weights.T)

    def updateWeights(self, inputs, learning_rate):
        self.weights += inputs.T.dot(self.delta) * learning_rate

In [47]:
class Network:
    def __init__(self, layers):
        self.layers = layers

    def forward(self, input):
        output = input
        for layer in self.layers:
            output = layer.forward(output)
        return output

    def backprop(self, inp, learning_rate, error):
        # Compute deltas at each layer starting from the last one
        for layer in reversed(self.layers):
            error = layer.computeGradient(error)

    # Update the weights
        for layer in self.layers:
            layer.updateWeights(inp, learning_rate)
            inp = layer.output
            
    def train(self, inputs, targets, epochs=50, learning_rate=0.01):
        errors = []
        for _ in range(epochs):
            for inp, target in zip(inputs, targets):
                output = self.forward(inp)
                error = (target - np.argmax(output))
                errors.append(error)
                self.backprop(inp, learning_rate, error)
            mse = (np.array(errors) ** 2).mean()
            print(mse)
                
            

In [20]:
wine = load_wine()

In [102]:
enc = OneHotEncoder()
wine.target = wine.target.reshape(-1, 1)
wine.target = enc.fit_transform(wine.target)
wine.target

/home/john/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


<178x3 sparse matrix of type '<class 'numpy.float64'>'
	with 178 stored elements in Compressed Sparse Row format>

In [106]:
wine.target = np.array(wine.target.todense())

In [107]:
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.2)

In [115]:
y_train = y_train.reshape(142, 1, 3)

In [122]:
mlp = nn.Network([nn.Layer(13, 40),
                 nn.Layer(40, 3)])

In [132]:
for epoch in range(800):
    errors = []
    accuracy = 1
    for input, target in zip(X_train, y_train):
    # Forward
        output = mlp.forward(input)

    # Compute the error
        error = target - output
        errors.append(error)

    # Back-propagate the error
        mlp.backprop(input, error, 0.1)

  # Compute the Mean Squared Error of all examples each 100 epoch
    if epoch % 10 == 0:
        mse = (np.array(errors) ** 2).mean()
        print("  Epoch %3d   MSE: %.3f" % (epoch, mse))
        print(f"{accuracy}")

        if mse <= target_mse:
            print("  * Target MSE reached *")
            break


  Epoch   0   MSE: 0.660
-141
  Epoch  10   MSE: 0.652
-138
  Epoch  20   MSE: 0.653
-141
  Epoch  30   MSE: 0.640
-138
  Epoch  40   MSE: 0.636
-137
  Epoch  50   MSE: 0.641
-139
  Epoch  60   MSE: 0.663
-141
  Epoch  70   MSE: 0.671
-141
  Epoch  80   MSE: 0.665
-141
  Epoch  90   MSE: 0.650
-141
  Epoch 100   MSE: 0.646
-140
  Epoch 110   MSE: 0.646
-140
  Epoch 120   MSE: 0.636
-141
  Epoch 130   MSE: 0.655
-141
  Epoch 140   MSE: 0.665
-141
  Epoch 150   MSE: 0.649
-141
  Epoch 160   MSE: 0.650
-141
  Epoch 170   MSE: 0.636
-141
  Epoch 180   MSE: 0.659
-141
  Epoch 190   MSE: 0.653
-138
  Epoch 200   MSE: 0.678
-141
  Epoch 210   MSE: 0.665
-141
  Epoch 220   MSE: 0.653
-141
  Epoch 230   MSE: 0.670
-141
  Epoch 240   MSE: 0.641
-136
  Epoch 250   MSE: 0.660
-141
  Epoch 260   MSE: 0.663
-140
  Epoch 270   MSE: 0.644
-140
  Epoch 280   MSE: 0.642
-140
  Epoch 290   MSE: 0.665
-141
  Epoch 300   MSE: 0.644
-141
  Epoch 310   MSE: 0.664
-141
  Epoch 320   MSE: 0.666
-141
  Epoch 33

(1, 1)